# Steps Analysis (02/17/17 - 10/24/18)

In [ ]:
from main_clean_function import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = main_clean_function('./data/export.xml')

In [ ]:
df.columns

In [ ]:
summed_df = df.groupby(by=['start_date', 'end_date']).sum()
summed_df['steps_per_hour'] = summed_df['num_steps']/summed_df['duration']
summed_df.reset_index(inplace=True)

In [ ]:
def drop_change_rename_df(df, x, col_rnm, drc=['start_time', 'end_time', 'source']):
    df.drop(index=[range(x-1)], inplace=True).reset_index()
    df['end_date'] = m_df['start_date'] - pd.Timedelta(x, unit='D')
    df.rename(columns={'num_steps': col_rnm+'num_steps', 'duration': col_rnm+'duration'})
    if type(drc) == list and len(drc) > 0:
        df.drop(columns=drc)
    return df

In [ ]:
def rolling_day_df(df, x):
    m_df = df.rolling(x).mean()
    md_df = df.rolling(x).median()
    std_df = df.rolling(x).std()
    s_df = df.rolling(x).sum()

    m_df = drop_change_rename_df(m_df, x, 'mean_')
    md_df = drop_change_rename_df(md_df, x, 'median_')
    std_df = drop_change_rename_df(std_df, x, 'std_')
    s_df = drop_change_rename_df(s_df, x, 'total_')
    
    merged_df = pd.merge(m_df, md_df, std_df, s_df)

In [ ]:
# rolling_day_df(summed_df, 30, mean)

In [ ]:
# summed_df.rolling(30).std()

In [ ]:
# summed_df.rolling(7).std().min()

In [ ]:
summed_df

In [ ]:
summed_df.groupby(summed_df['start_date'].dt.strftime('%b%y'), sort=False).mean()

In [ ]:
summed_df.num_steps.std()

In [ ]:
print_remainder(df)

In [ ]:
summed_df.tail(60)

In [ ]:
X = summed_df.start_date.values
y = df.num_steps.values
type(y)

In [ ]:
summed_df.plot(x='start_date', y='num_steps', kind='area')

In [ ]:
summed_df.min()